In [ ]:
!pip install fastapi
!pip install uvicorn
!pip install pickle5
!pip install pydantic
!pip install scikit-learn
!pip install requests
!pip install pypi-json
!pip install pyngrok==4.1.1

In [ ]:
!pip install python-multipart
!pip install nest-asyncio
!pip install aiofiles
!pip install ar-corrector

In [1]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import json
import uvicorn
from  pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
import os
import sys
from fastapi import FastAPI, File, UploadFile
from tempfile import NamedTemporaryFile
from fastapi.responses import HTMLResponse, StreamingResponse

In [2]:
sys.path.insert(0, '/content/folder')
from preprocessing import preprocess_sample
from config import args
from models.visual_frontend import VisualFrontend
from spell_sentence import Spell
import torch

In [3]:
app = FastAPI()

In [4]:
gpuAvailable = torch.cuda.is_available()
device = torch.device("cuda" if gpuAvailable else "cpu")

# RUN DEMO


In [5]:
from config import args
from models.video_net import VideoNet
from models.visual_frontend import VisualFrontend
from models.lrs2_char_lm import LRS2CharLM
from utils import prepare_main_input, collate_fn
from preprocessing import preprocess_sample
from decoders import ctc_greedy_decode, ctc_search_decode
from demo import demo_test

# RUN preprocessing

In [ ]:
vf = VisualFrontend()
vf.load_state_dict(torch.load(args["TRAINED_FRONTEND_FILE"], map_location=device))
vf.to(device)

In [ ]:
params = {"roiSize":args["ROI_SIZE"], "normMean":args["NORMALIZATION_MEAN"], "normStd":args["NORMALIZATION_STD"], "vf":vf}

# RUN app

In [6]:
import shutil
from fastapi.concurrency import run_in_threadpool
import aiofiles
import asyncio
import os
import binascii

In [7]:
from typing_extensions import Annotated
from starlette.requests import Request

In [8]:
from moviepy.editor import VideoFileClip
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import math
def cut_video(video_path):
  all_paths=[]
  clip = VideoFileClip("/content/folder/output.mp4")
  #print((clip.duration))
  duration       = (clip.duration)
  fps            = clip.fps
  #print((duration))
  i=0
  while(i<=int(duration) and i != int(duration)):
    videos = str(i+1)+".mp4"
    all_paths.append("/content/"+videos)
    #print(i)
    #print(i+2)
    ffmpeg_extract_subclip("/content/folder/output.mp4", i,i+2,targetname=videos)
    i=i+2
  return all_paths

In [9]:
@app.post("/uploadfiles/")
async def create_upload_file(request: Request):
    rs = ""
    data: bytes = await request.body()
    output = data
    print(output)
    #video_path = binascii.unhexlify(output)
    #print(video_path)
    f = open("/content/folder/output.mp4", 'wb')
    f.write(output)
    f.close()
    cutting_video=cut_video("/content/folder/output.mp4")
    for video in cutting_video:
      try:
        rs = rs+demo_test(video)+" "
      finally:
        os.unlink(video)
    grid="/content/folder/words75.txt"
    Sp = Spell(grid)
    pred=Sp.sentence(rs.strip())  
    return pred

In [10]:
auth_token = "2OKzCE2QibAxS9SV8WQ6cjmTRxk_2nP26rKamD6rFb8yx1Ufq" #@param {type:"string"}
# Since we can't access Colab notebooks IP directly we'll use
# ngrok to create a public URL for the server via a tunnel

# Authenticate ngrok
# https://dashboard.ngrok.com/signup
# Then go to the "Your Authtoken" tab in the sidebar and copy the API key
import os
os.system(f"ngrok authtoken {auth_token}")

0

In [11]:
# Create tunnel
public_url = ngrok.connect( port='8000')

In [12]:
# Check if it exists
!ps aux | grep ngrok

root       36852 26.0  0.1 726760 24700 ?        Sl   12:51   0:00 /usr/local/lib/python3.10/dist-packages/pyngrok/bin/ngrok start --none --log=stdout
root       36866  0.0  0.0   6904  3292 ?        S    12:51   0:00 /bin/bash -c ps aux | grep ngrok
root       36868  0.0  0.0   6444   720 ?        S    12:51   0:00 grep ngrok


In [ ]:
import nest_asyncio

# Allow for asyncio to work within the Jupyter notebook cell
nest_asyncio.apply()

import uvicorn

# Run the FastAPI app using uvicorn
print(public_url)
uvicorn.run(app)

In [ ]:
# Kill tunnel
ngrok.disconnect(public_url=public_url)

In [ ]:
!zip -r /content/drive/MyDrive/final/End_Api2.zip /content/folder